In [31]:
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder

import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'..')))
from utils import load_tensor
from models import BiLSTM, CNN_BiLSTM
from evaluation import calculate_confusion_matrix, class_accuracy, class_f1_score

In [32]:
def remap_targets(target_tensor: torch.TensorType, old_le, new_le):
    inverse_tensor = old_le.inverse_transform(target_tensor.long())
    for idx, label in enumerate(inverse_tensor):
        if label == 'ARG_RESPONDENT' or label == 'ARG_PETITIONER':
            inverse_tensor[idx] = "ARG"
        elif label == 'PRE_NOT_RELIED' or label == 'PRE_RELIED':
            inverse_tensor[idx] = 'PRE'
    new_tensor = torch.tensor(new_le.transform(inverse_tensor))
    return new_tensor

In [33]:
list_of_targets_old = ['ISSUE', 'FAC', 'NONE', 'ARG_PETITIONER', 'PRE_NOT_RELIED', 'STA', 'RPC', 'ARG_RESPONDENT', 'PREAMBLE', 'ANALYSIS', 'RLC', 'PRE_RELIED', 'RATIO']
label_encoder_old = LabelEncoder().fit(list_of_targets_old)
list_of_targets_new = ['ISSUE', 'FAC', 'NONE', 'ARG', 'PRE', 'STA', 'RPC', 'PREAMBLE', 'ANALYSIS', 'RLC', 'RATIO']
label_encoder_new = LabelEncoder().fit(list_of_targets_new)

In [34]:
sample_input, sample_target = None, None
for idx in range(246):
    if sample_input is None:
        sample_input = load_tensor(filepath=f"../train_document/doc_{idx}/embedding")
        sample_target = load_tensor(filepath=f"../train_document/doc_{idx}/label")
    else:
        sample_input = torch.cat((sample_input,load_tensor(filepath=f"../train_document/doc_{idx}/embedding")), dim=0)
        sample_target = torch.cat((sample_target,load_tensor(filepath=f"../train_document/doc_{idx}/label")), dim=0)

In [35]:
sample_target.size()

torch.Size([28864])

In [36]:
remapped_target = remap_targets(sample_target, label_encoder_old, label_encoder_new)

In [37]:
from sklearn.utils.class_weight import compute_class_weight
    
class_weights = compute_class_weight(class_weight = "balanced",
                                    classes = np.unique(remapped_target.numpy()),
                                    y = remapped_target.numpy())
class_weights = torch.FloatTensor(class_weights)
class_weights

tensor([0.2460, 1.3107, 0.4584, 7.1499, 1.8544, 1.6639, 0.6347, 3.8990, 3.5033,
        2.4341, 5.4895])

# BERT-Base

## BiLSTM

In [44]:
model1 = BiLSTM(hidden_size=128, dropout= 0.25, output_size= 11)
optimizer = torch.optim.Adam(model1.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model1.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model1(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.05:
        break
# batch_loss.append(loss.item())


-----------------------------------------Starting Training------------------------------------------


  0%|          | 0/246 [00:00<?, ?it/s]

100%|██████████| 246/246 [00:08<00:00, 27.83it/s]


Epoch: 1 	 Loss: 1.43475


100%|██████████| 246/246 [00:06<00:00, 40.76it/s]


...

Epoch: 49 	 Loss: 0.08222


100%|██████████| 246/246 [00:09<00:00, 24.92it/s]


Epoch: 50 	 Loss: 0.05691


100%|██████████| 246/246 [00:09<00:00, 26.51it/s]

Epoch: 51 	 Loss: 0.04955


In [45]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model1, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))
# result.append((config, (average_accuracy, average_f1)))

Accuracies: [0.84980237 0.39393939 0.7751938  0.78       0.92857143 0.45962733
 0.98434442 0.54878049 0.60416667 0.85555556 0.47368421] 
 Average acccuracy: 0.6957877877225896
F1 Scores: [0.74394459 0.44067792 0.81699341 0.77999995 0.9086021  0.62447253
 0.99113295 0.58441553 0.35365849 0.86516849 0.53731338] 
 Average F1: 0.6951253954753968


## CNN-BiLSTM

In [46]:
model2 = CNN_BiLSTM(hidden_size=128, dropout= 0.25, output_size= 11)
optimizer = torch.optim.Adam(model2.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model2.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model2(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.05:
        break
# batch_loss.append(loss.item())

-----------------------------------------Starting Training------------------------------------------


  0%|          | 0/246 [00:00<?, ?it/s]

100%|██████████| 246/246 [00:18<00:00, 13.05it/s]


Epoch: 1 	 Loss: 1.56903


100%|██████████| 246/246 [00:09<00:00, 26.63it/s]


...

Epoch: 61 	 Loss: 0.07679


100%|██████████| 246/246 [00:06<00:00, 38.90it/s]


Epoch: 62 	 Loss: 0.05715


100%|██████████| 246/246 [00:06<00:00, 39.18it/s]

Epoch: 63 	 Loss: 0.04668


In [47]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model2, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

Accuracies: [0.83424808 0.51351351 0.78187404 0.73584906 0.94505495 0.58947368
 0.99405941 0.47524752 0.58928571 0.86170213 0.5       ] 
 Average acccuracy: 0.7109370991812456
F1 Scores: [0.80593844 0.42696624 0.82764223 0.7572815  0.92473113 0.65497071
 0.99504455 0.55491325 0.38372089 0.89010984 0.59154925] 
 Average F1: 0.7102607293865731


# LEGAL-BERT

## BiLSTM

In [48]:
model3 = BiLSTM(hidden_size=128, dropout= 0.25, output_size= 11)
optimizer = torch.optim.Adam(model3.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model3.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}_legal/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}_legal/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model3(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.1:
        break

-----------------------------------------Starting Training------------------------------------------


  1%|          | 2/246 [00:00<00:16, 14.64it/s]

100%|██████████| 246/246 [00:06<00:00, 40.87it/s]


Epoch: 1 	 Loss: 2.23895


100%|██████████| 246/246 [00:05<00:00, 46.04it/s]


...

Epoch: 98 	 Loss: 0.18572


100%|██████████| 246/246 [00:05<00:00, 44.88it/s]


Epoch: 99 	 Loss: 0.17958


100%|██████████| 246/246 [00:05<00:00, 43.83it/s]

Epoch: 100 	 Loss: 0.18010


In [49]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}_legal/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}_legal/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model3, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

Accuracies: [0.62295082 0.05357143 0.58757962 0.30909091 0.70833333 0.25
 0.75136116 0.25714286 0.25       0.4609375  0.5       ] 
 Average acccuracy: 0.43190614792450505
F1 Scores: [0.70744676 0.03749995 0.61143326 0.32380947 0.38931294 0.01282051
 0.78483407 0.25352108 0.03225805 0.54629625 0.17142854] 
 Average F1: 0.35187826173658254


## CNN-BiLSTM

In [50]:
model4 = CNN_BiLSTM(hidden_size=128, dropout= 0.25, output_size= 11)
optimizer = torch.optim.Adam(model4.parameters(), lr= 5e-4)
loss_function = nn.CrossEntropyLoss(weight= class_weights)

print(f'{"Starting Training":-^100}')
model4.train()
loss_list = []
for epoch in range(100):
    running_loss = 0
    for idx in tqdm(range(246)):
        TRAIN_emb = load_tensor(filepath=f"../train_document/doc_{idx}_legal/embedding")
        TRAIN_labels = load_tensor(filepath=f"../train_document/doc_{idx}_legal/label")
        TRAIN_labels = remap_targets(TRAIN_labels, label_encoder_old, label_encoder_new)
        if TRAIN_emb.size(0) == 0:
            continue
        output = model4(TRAIN_emb)
        loss = loss_function(output,TRAIN_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    # scheduler.step()
    # scheduler1.step()
    # scheduler2.step()
    # running_lr.append(model_opt.state_dict()['param_groups'][0]['lr'])
    loss_list.append(running_loss/246)
    print(f"Epoch: {epoch+1} \t Loss: {running_loss/246:.5f}")
    if running_loss/246 < 0.1:
        break

-----------------------------------------Starting Training------------------------------------------


  1%|          | 2/246 [00:00<00:14, 17.37it/s]

100%|██████████| 246/246 [00:11<00:00, 22.25it/s]


Epoch: 1 	 Loss: 2.23016


100%|██████████| 246/246 [00:08<00:00, 28.18it/s]


...

Epoch: 98 	 Loss: 0.10374


100%|██████████| 246/246 [00:06<00:00, 37.36it/s]


Epoch: 99 	 Loss: 0.10235


100%|██████████| 246/246 [00:07<00:00, 31.69it/s]

Epoch: 100 	 Loss: 0.10181


In [51]:
cm = None
for i in range(29):
    TEST_emb = load_tensor(filepath=f"../test_document/doc_{i}_legal/embedding")
    TEST_labels = load_tensor(filepath=f"../test_document/doc_{i}_legal/label")
    TEST_labels = remap_targets(TEST_labels, label_encoder_old, label_encoder_new)
    conf_matrix_helper = calculate_confusion_matrix(TEST_emb, TEST_labels, model4, num_labels= 11)
    if cm is None:
        cm = conf_matrix_helper
    else:
        cm = np.add(cm, conf_matrix_helper)
        
accuracies = class_accuracy(cm)
f1_scores = class_f1_score(cm)
average_accuracy = np.mean(accuracies)
average_f1 = np.mean(f1_scores)

print("Accuracies: {} \n Average acccuracy: {}".format(accuracies, average_accuracy))
print("F1 Scores: {} \n Average F1: {}".format(f1_scores, average_f1))

Accuracies: [0.68943436 0.09677419 0.52981651 0.35294118 0.51908397 0.10769231
 0.84485981 0.30909091 0.20689655 0.55208333 0.        ] 
 Average acccuracy: 0.382606648469596
F1 Scores: [0.67573217 0.04444441 0.63680216 0.28571424 0.42367596 0.09929073
 0.87006732 0.26771649 0.08275859 0.57608691 0.        ] 
 Average F1: 0.36020808798010634
